## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow_addons.optimizers import AdamW, Lookahead
from tensorflow.keras.layers import Concatenate, Add
from tensorflow.keras.layers import Activation, Input
from tensorflow.keras.layers import Embedding, Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout

## Prepare data for model training

In [2]:
with open("../input/tps-june-data-preprocess/TPS_June_Dataset_Set1.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

col_list = test_df.columns.to_list()

del processed_data
gc.collect()

42

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (200000, 75) 
Ytrain: (200000,) 
Ytrain_oh: (200000, 9) 
Xtest: (100000, 75)


0

## Build the model

In [4]:
def dnn_model(n_features):
    
    x_input = Input(shape=(n_features,))
    
    x = Embedding(512, 32)(x_input)
    
    x = Conv1D(filters=64, kernel_size=3, 
               strides=2, padding='same', 
               kernel_regularizer=l2(0.0003),
               kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    x = Conv1D(filters=96, kernel_size=3, 
               strides=2, padding='same', 
               kernel_regularizer=l2(0.0003),
               kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.15)(x)
    
    x = Dense(units=32, kernel_initializer='he_uniform', 
                kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dropout(rate=0.2)(x)
    
    x = Dense(units=16, kernel_initializer='he_uniform', 
                kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dropout(rate=0.1)(x)

    x_output = Dense(units=9, activation='softmax', 
                     kernel_initializer='he_uniform')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

In [5]:
model = dnn_model(Xtrain.shape[1])
model.compile(loss='categorical_crossentropy',
              optimizer=Lookahead(AdamW(lr=1e-2, 
                                        weight_decay=1e-5, 
                                        clipvalue=700), 
                                  sync_period=10))
model.summary()

Model: "DNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 75)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 75, 32)            16384     
_________________________________________________________________
conv1d (Conv1D)              (None, 38, 64)            6208      
_________________________________________________________________
batch_normalization (BatchNo (None, 38, 64)            256       
_________________________________________________________________
activation (Activation)      (None, 38, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 19, 96)            18528     
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 96)            38

In [6]:
FOLD = 10
NUM_SEED = 3
VERBOSE = 0

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_dnn = np.zeros((Xtrain.shape[0], 9))
y_pred_final_dnn = np.zeros((Xtest.shape[0], 9))
counter = 0
mini_batch_size = 128


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
        val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

        model = dnn_model(Xtrain.shape[1])
        model.compile(loss='categorical_crossentropy',
                      optimizer=Lookahead(AdamW(lr=1e-2, 
                                                weight_decay=1e-5, 
                                                clipvalue=700), 
                                          sync_period=10))

        early = EarlyStopping(monitor="val_loss", mode="min", 
                              restore_best_weights=True, 
                              patience=7, verbose=VERBOSE)

        reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                                      min_lr=1e-6, patience=3, 
                                      verbose=VERBOSE, mode='min')

        chk_point = ModelCheckpoint('./DNN_model.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')
        
        history = model.fit(
            train_x, train_y_oh, 
            batch_size=mini_batch_size,
            epochs=250, 
            verbose=VERBOSE, 
            workers=5,
            callbacks=[reduce_lr, early, chk_point], 
            validation_data=(val_x, val_y_oh)
        )
        
        model = load_model('./DNN_model.h5')

        y_pred = model.predict(val_x)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model.predict(Xtest)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn = y_pred_meta_dnn / float(NUM_SEED)
y_pred_final_dnn = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

Seed-24 | Fold-0 | OOF Score: 1.745411545256432
Seed-24 | Fold-1 | OOF Score: 1.7447170224899893
Seed-24 | Fold-2 | OOF Score: 1.7426175617660862
Seed-24 | Fold-3 | OOF Score: 1.7467244232651777
Seed-24 | Fold-4 | OOF Score: 1.7395248644541483
Seed-24 | Fold-5 | OOF Score: 1.7405364717219955
Seed-24 | Fold-6 | OOF Score: 1.7413497349957003
Seed-24 | Fold-7 | OOF Score: 1.7434011591174174
Seed-24 | Fold-8 | OOF Score: 1.743254952630587
Seed-24 | Fold-9 | OOF Score: 1.7464197778678034

Seed: 24 | Aggregate OOF Score: 1.743395751356534


Seed-3 | Fold-0 | OOF Score: 1.752987381762825
Seed-3 | Fold-1 | OOF Score: 1.7394856353411452
Seed-3 | Fold-2 | OOF Score: 1.742551665415708
Seed-3 | Fold-3 | OOF Score: 1.746928280158993
Seed-3 | Fold-4 | OOF Score: 1.7492070696601645
Seed-3 | Fold-5 | OOF Score: 1.73846428587439
Seed-3 | Fold-6 | OOF Score: 1.7422300477524755
Seed-3 | Fold-7 | OOF Score: 1.7388758250739424
Seed-3 | Fold-8 | OOF Score: 1.745873169744201
Seed-3 | Fold-9 | OOF Score: 1.73

In [7]:
np.savez_compressed('./DNN_Meta_Features.npz',
                    y_pred_meta_dnn=y_pred_meta_dnn, 
                    oof_score=oof_score,
                    y_pred_final_dnn=y_pred_final_dnn)

## Create submission file

In [8]:
test_df = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_dnn[:,1]
submit_df['Class_2'] = y_pred_final_dnn[:,2]
submit_df['Class_3'] = y_pred_final_dnn[:,3]
submit_df['Class_4'] = y_pred_final_dnn[:,4]
submit_df['Class_5'] = y_pred_final_dnn[:,5]
submit_df['Class_6'] = y_pred_final_dnn[:,6]
submit_df['Class_7'] = y_pred_final_dnn[:,7]
submit_df['Class_8'] = y_pred_final_dnn[:,8]
submit_df['Class_9'] = y_pred_final_dnn[:,0]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.064162,0.392750,0.152761,0.027778,0.013738,0.159395,0.022114,0.045383,0.121919
1,200001,0.043371,0.069697,0.053390,0.022357,0.015213,0.284926,0.082513,0.298607,0.129925
2,200002,0.021718,0.031089,0.022415,0.010352,0.006404,0.709791,0.029036,0.116828,0.052366
3,200003,0.049064,0.136290,0.094643,0.031516,0.019839,0.215220,0.076603,0.215464,0.161361
4,200004,0.045715,0.118192,0.082658,0.028216,0.017646,0.270401,0.072014,0.216537,0.148621


In [9]:
submit_df.to_csv("./DNN_submission.csv", index=False)